# Radiology Report Analysis with Ollama (Local Llama 3.1 8B)

This notebook demonstrates how to use Ollama running locally to analyze MIMIC-IV radiology reports for pulmonary nodules according to Fleischner guidelines.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
from src.services.ollama_llm import OllamaClient

In [ ]:
# Initialize Ollama client
# Make sure Ollama is running: ollama serve
# And model is pulled: ollama pull llama3.1:8b

client = OllamaClient()
print(f"Using model: {client.model_name}")

In [ ]:
# Load MIMIC-IV radiology data
df = pd.read_csv('/Users/amulyaveldandi/Downloads/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/note/radiology.csv')
print(f"Total radiology reports: {len(df)}")
df.head()

In [ ]:
# Filter for CT Chest reports with pulmonary nodules
ct_chest = df[df['text'].str.contains('CT CHEST|CT OF THE CHEST|CHEST CT', case=False, na=False)]
print(f"Total CT Chest reports: {len(ct_chest)}")

nodule_terms = 'nodule|nodular|pulmonary nodule'
ct_chest_nodules = ct_chest[ct_chest['text'].str.contains(nodule_terms, case=False, na=False)]
print(f"CT Chest reports with nodules: {len(ct_chest_nodules)}")

In [ ]:
# Get a sample report
if len(ct_chest_nodules) > 0:
    sample_report = ct_chest_nodules.iloc[0]['text']
    print("Sample CT Chest report with nodule:")
    print("=" * 80)
    print(sample_report[:1000])

In [ ]:
# Use Ollama to extract nodule information
prompt = f"""
Analyze this radiology report and extract pulmonary nodule information:

{sample_report}

Extract the following information:
1. Number of nodules mentioned
2. Size of each nodule (in mm)
3. Location of nodules
4. Nodule characteristics (solid, ground-glass, part-solid)
5. Any follow-up recommendations mentioned

Provide a concise summary.
"""

print("Analyzing report with Llama 3.1 8B...\n")
response = client.generate(prompt, temperature=0.1, max_tokens=512)
print(response)

In [ ]:
# Extract structured JSON with Fleischner guideline recommendations
fleischner_prompt = f"""
Analyze this CT chest report for pulmonary nodules and provide Fleischner guideline recommendations:

{sample_report}

Return a JSON object with:
{{
  "nodules_found": true/false,
  "nodule_count": number,
  "largest_nodule_size_mm": number or null,
  "nodule_type": "solid" or "ground-glass" or "part-solid" or "mixed" or null,
  "fleischner_applicable": true/false,
  "recommended_followup": "string describing follow-up interval",
  "confidence": "high" or "medium" or "low"
}}
"""

print("Extracting structured data...\n")
json_response = client.generate_json(fleischner_prompt)
print(json.dumps(json_response, indent=2))

In [ ]:
# Batch process multiple reports
def analyze_nodule_report(report_text: str) -> dict:
    """Analyze a radiology report for nodule information."""
    prompt = f"""
Analyze this radiology report for pulmonary nodules:

{report_text[:2000]}  # Limit to first 2000 chars

Return JSON with:
{{
  "has_nodules": true/false,
  "nodule_size_mm": number or null,
  "followup_needed": true/false
}}
"""
    try:
        return client.generate_json(prompt)
    except Exception as e:
        return {"error": str(e)}

# Process first 5 reports (you can increase this)
sample_size = 5
results = []

print(f"Processing {sample_size} reports...\n")
for idx, row in ct_chest_nodules.head(sample_size).iterrows():
    print(f"Processing report {idx}...")
    result = analyze_nodule_report(row['text'])
    result['note_id'] = row['note_id']
    results.append(result)

# Convert to DataFrame
results_df = pd.DataFrame(results)
print("\nResults:")
results_df

In [ ]:
# Check Ollama client metrics
print(f"Total tokens used: {client.total_tokens}")
print(f"Total API calls: {client._total_calls}")
print(f"Error rate: {client.error_rate:.2%}")
if client.latencies_ms:
    print(f"Average latency: {sum(client.latencies_ms)/len(client.latencies_ms):.2f}ms")